In [3]:
import sys
sys.path.append('../')

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import SubsetRandomSampler
import numpy as np
import random

import numpy as np
import pandas as pd
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import sklearn

eps = np.finfo(float).eps

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [23]:
batch_size = 4
feat_dim = 512
tau = 1
x_i = torch.randn(batch_size, feat_dim)
x_j = x_i + 0.05 * torch.randn(batch_size, feat_dim)
x = torch.cat((x_i, x_j), dim=0)

In [22]:
sim_mat_nom = torch.mm(x, x.T)
sim_mat_denom = torch.mm(torch.norm(x, dim=1).unsqueeze(1), torch.norm(x, dim=1).unsqueeze(1).T)

sim_mat = sim_mat_nom / sim_mat_denom.clamp(min=1e-16)
sim_mat = torch.exp(sim_mat / tau)

In [ ]:
print(sim_mat)